In [ ]:
import torch
import json
from transformers import AutoTokenizer
from transformer_lens import HookedTransformer
import numpy as np
from typing import List, Tuple, TypeAlias

from dotenv import load_dotenv
import os
import goodfire
from utils import get_completion

from utils import get_activations
from utils import compute_steering_vector
from utils import apply_steering

In [ ]:
prompt_template = "I am {}. What advice do you have for me?"

## Create contrastive dataset using Goodfire API

In [ ]:
load_dotenv()
api_key = os.getenv("GOODFIRE_API_KEY")
client = goodfire.Client(api_key)
variant = goodfire.Variant("meta-llama/Meta-Llama-3-8B-Instruct")

features, relevance = client.features.search(
    "deception",
    model=variant,
    top_k=5
)
print(features)

chosen_feature = features[3]
print(chosen_feature)

In [ ]:
scenarios = [
    'playing a game of diplomacy',
    'applying for a job',
    'participating in a poker tournament',
    'performing as a magician',
    'coaching a sports team',
    'negotiating a business deal',
    'acting in a theater production',
    'planning a surprise party',
    'playing capture the flag',
    'writing mystery fiction',
    'doing improvisational comedy',
    'designing escape rooms',
    'competing in a chess tournament',
    'playing paintball with friends',
    'participating in debate club',
    'leading a scavenger hunt',
    'doing stage magic tricks',
    'playing hide and seek',
    'writing riddles for an event',
    'organizing team building games',
    'competing in martial arts',
    'playing laser tag',
    'running an escape room business',
    'doing improv theater games',
    'playing social deduction games',
    'teaching strategy games',
    'hosting a murder mystery party',
    'practicing negotiation skills',
    'playing card games',
    'training for sports competitions',
    'writing mystery novels',
    'designing puzzle games',
    'competing in a trivia contest',
    'playing charades',
    'participating in mock trials',
    'planning an elaborate proposal',
    'doing standup comedy',
    'practicing sleight of hand',
    'playing strategic board games',
    'running dungeons and dragons',
    'playing werewolf/mafia games',
    'competing in spelling bees',
    'doing geocaching',
    'practicing tactical sports',
    'organizing treasure hunts',
    'playing strategy video games',
    'hosting quiz shows',
    'practicing mental sports',
    'doing puzzle competitions',
    'playing capture the flag online',
    'teaching game theory',
    'running simulation games'
]

variant.reset()
variant.set(chosen_feature, 0.6, mode="nudge")
positive_nudge_completions: list[str] = []
for scenario in scenarios:
    prompt = prompt_template.format(scenario)
    completion = get_completion(client, variant, prompt)
    positive_nudge_completions.append(completion)
    # print(completion)

variant.reset()
variant.set(chosen_feature, -0.3, mode="nudge")
negative_nudge_completions: list[str] = []
for scenario in scenarios:
    prompt = prompt_template.format(scenario)
    completion = get_completion(client, variant, prompt)
    negative_nudge_completions.append(completion)


In [ ]:
# create dictionary containing scenarios and completions
data = {}
for i, scenario in enumerate(scenarios):
    data[scenario] = {
        "positive_nudge": positive_nudge_completions[i],
        "negative_nudge": negative_nudge_completions[i]
    }

# save dictionary to json file
with open('data.json', 'w') as f:
    json.dump(data, f)

## Create steering vector, using contrastive dataset

In [ ]:
# load the data
with open('data02.json') as f:
    data = json.load(f)

# create full list of contrastive pairs from the data
pairs = []
for scenario, completions in data.items():
    user_content = prompt_template.format(scenario)
    positive_nudge = completions['positive_nudge']
    negative_nudge = completions['negative_nudge']
    pairs.append((
        [{"role": "user", "content": user_content}, {"role": "assistant", "content": positive_nudge}],
        [{"role": "user", "content": user_content}, {"role": "assistant", "content": negative_nudge}]
    ))

pairs[37]

In [ ]:
# Compute steering vector
model = HookedTransformer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

In [ ]:
steering_vector = compute_steering_vector(model, pairs[0:1], layer=19)

In [ ]:
print(steering_vector.max())
print(steering_vector.min())
print(steering_vector.mean())

In [ ]:
scenarios = list(data.keys())
scenario = scenarios[0]

# Apply steering to new text
messages = [
    {"role": "user", "content": prompt_template.format(scenario)},
]

for strength in [-1, -0.5, 0, 0.1, 0.3, 1]:
    modified_text = apply_steering(model, messages, steering_vector, n_tokens=25, layer=19, strength=strength)
    print(f"Strength: {strength}")
    print(modified_text)
    print()
    print()
    print('='*20)

## Understanding `apply_chat_template`

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
    {"role": "assistant", "content": "I be a pirate chatbot, arrr!"},
    {"role": "user", "content": "What do you do?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
)

In [ ]:
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=False)
print(decoded_text)

In [ ]:
input_ids

In [ ]:
tokenizer.encode(decoded_text, add_special_tokens=False)

In [ ]:
for i in [9125, 882, 78191, 271]:
    print(i, tokenizer.decode([i], skip_special_tokens=False))

for i in range(-10, 25):
    print(i, tokenizer.decode([128000+i], skip_special_tokens=False))